In [4]:
#Install the libraries

!pip install --upgrade --quiet transformers datasets evaluate fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [13]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import evaluate
import torch
import re

In [2]:
#Download the dataset

dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [9]:
#Print  random reviews

sample_texts = [dataset["train"][i]["text"] for i in random.sample(range(25000), 5)]

for i, text in enumerate(sample_texts):
    print(f"--- Sample {i+1} ---
{text[:500]}
")

--- Sample 1 ---
My flatmate rented out this film the other night, so we watched it together.<br /><br />The first impression is actually a positive one, because the whole movie is shot in this colorful, grainy, post-MTV texture. Fast sequences, cool angles, sweeping camera moves - for the moment there you feel like you about to watch another "Snatch", for the moment....<br /><br />When the plot actually starts unfolding, one starts to feel as if one over-dosed amphetamine. things just don't make sense anymore. 

--- Sample 2 ---
Throughly enjoy all the musical numbers each time I see this movie. Never seem to tire of it. Fred and Ginger are always a pleasure to watch. Seeing "Lucy" and Betty Grable before they hit the big time, is fun to watch.

--- Sample 3 ---
Opening credits: great. Music: just right for this film. Cinematography: sleazy to great effect.<br /><br />Harrowing excitement in a train-wreck sort of way. This is how out of control some lives can become. "Wonderland" depi

Preprocessing

In [3]:
#Remove empty and short reviews

def rem_empty(example):
    return len(example["text"].strip()) > 20

dataset["train"] = dataset["train"].filter(rem_empty)
dataset["test"] = dataset["test"].filter(rem_empty)

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [4]:
#Removing unnecessary spaces

def clean_text(example):
    example["text"] = example["text"].strip()
    return example

dataset = dataset.map(clean_text)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
#Remove HTML break tags and replace with space

def clean_text(example):
    text = example["text"]
    text = re.sub(r"<br\s*/?>", " ", text)
    example["text"] = text
    return example

dataset = dataset.map(clean_text)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
#Tokenization

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_fun(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(preprocess_fun, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Prepare data for Training

In [14]:
#Split dataset into training and test

train_data = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_data = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [15]:
#Loading BERT model

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
